# Smart Mendr

We propose Smart Mendr, a new classification Model that applies Ensemble Learning and Data-driven Rectification to handle both scenarios of inaccurate and incomplete supervision. An overview of the proposed method is illustrated in Figure 5.1. As the figure shows, the method has two phases. In the first phase, Smart Mendr applies a preliminary stage of ensemble learning to estimate the probability of each instance being mislabeled and produce initially weak labels for unlabeled data. However, to overcome the challenges of noise detection using ensemble learning, we apply a semi-supervised learning approach to combine the output of the ensemble and report the noisy points. After that, the proposed method, in the second phase, applies a smart correcting procedure using meta-active learning to provide true labels for both noisy and unlabeled points

In [30]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import warnings
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reading and preparing the data from csv file

In [31]:
def load_data(input_folder, dataset_name, y_column):
    p_path =  input_folder + "//" + dataset_name + "_noisy.csv"
    u_path =  input_folder + "//" + dataset_name + "_unlabeled.csv"
    original_path = input_folder + "//" + dataset_name + ".csv"
    original_data = pd.read_csv(original_path, sep=',')

    df_p = pd.read_csv(p_path, sep=',')
    ground_truth = df_p[y_column]
    df_p = df_p.drop(y_column, axis = 1)
    df_u = pd.read_csv(u_path, sep=',') 
    df_u = df_u.drop(y_column, axis = 1)

    return df_p, df_u, ground_truth, original_data

In [32]:
df_p, df_u, ground_truth, original_data = load_data("datasets", "Banana", "prediction")


DU = original_data
DU.index = [x for x in range(1, len(DU)+1)]
DU.index.name = 'index'
obj_columns = DU.select_dtypes(['object']).columns    
for col in obj_columns:
    DU[col] = DU[col].astype('category')
cat_columns = DU.select_dtypes(['category']).columns    
DU[cat_columns] = DU[cat_columns].apply(lambda x: x.cat.codes)

obj_columns = df_p.select_dtypes(['object']).columns    
for col in obj_columns:
    df_p[col] = df_p[col].astype('category')
cat_columns = df_p.select_dtypes(['category']).columns    
df_p[cat_columns] = df_p[cat_columns].apply(lambda x: x.cat.codes)


obj_columns = df_u.select_dtypes(['object']).columns   
for col in obj_columns:
    df_u[col] = df_u[col].astype('category')
cat_columns = df_u.select_dtypes(['category']).columns    
df_u[cat_columns] = df_u[cat_columns].apply(lambda x: x.cat.codes)

## Ensemble Learning
In this phase, the proposed method aims at detecting data points with noisy labels in Dp and producing initial labels for the unlabeled points in Du. Therefore, the phase employs a set of ensembles in two stages. In the first stage, a set of base learners are built to produce predictions for the data points in D. Then, the ensemble predictor is utilized in the second stage to detect noisy points Dnoise in Dp.

In [33]:
import numpy as np
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(df_p, ground_truth)
predictions_p = clf.predict(df_u)
predictions_u = clf.predict(df_p)
predictions = np.concatenate((predictions_u, predictions_p))

original_p = ground_truth
original_u = np.zeros(df_u.shape[0])
original = np.concatenate((original_u, original_p))

In [34]:
from scipy import sparse

S = np.column_stack((predictions,original))
S = S.astype(int)
L_train= sparse.csr_matrix(S) 


In [35]:
S.shape

(5297, 2)

Finally, to combine the output of the ensemble_learner and generate an initial vector of probabilistic labels, we employ a generative model to learn the accuracies of the predictions in H_Best and produce a single probabilistic label for each data point in the unlabeled dataset. 

In [36]:
import smartmendr.ensemble_learner
from smartmendr.ensemble_learner.util_ensemble import *

global gen_model 
gen_model, gen_train_marginals = Fitting_Gen_Model(L_train)

Inferred cardinality: 2
[1.54208115 0.14582523]
0.0


In [37]:
predictions = gen_model.predictions(L_train, batch_size=None)
gen_model.learned_lf_stats()

,Accuracy,Coverage,Precision,Recall
0,0.955539,0.8277,0.957265,0.793232
1,0.567414,0.6690,0.577388,0.375762


In [38]:
abstain = 0
agreements_list=[]
for rows in L_train:
    if len(rows.data) == 0:
        abstain=abstain+1      
    agreements_list.append(abs(sum(rows.data)))

## Data-driven Active Learner
The component tries to integrate the user in the loop at this point by employing active learning. However, our problem settings do not impose traditional active learning scenarios where we usually have a small set of labeled points and a larger set of unlabeled data. Instead, we deal with a set of probabilistic labels that are classified based on the confidence of the generative model. Therefore, we adopt meta-active learning in this component and apply a data-driven approach to learn the query strategy. The approach formulates the process of designing the query strategy as a regression problem. We train a regression model to predict the reduction of the generalization error associated with adding a labeled point {xi, yi} to the training data of a classifier. Our main hypothesis is that this regressor can serve as the query strategy in our problem settings to outperform the baseline strategies since it is customized to the underlying distribution and considers the output of the generative model.

In [39]:
index_list = []
agreements_list=[]
unlabeled_list=[]
i = 1
for rows in L_train:
    index_list.append(i)
    if len(rows.data) == 0:
        unlabeled_list.append(True)
    else:
        unlabeled_list.append(False)
    agreements_list.append(abs(sum(rows.data)))
    i=i+1
    
df_with_additional_info = pd.DataFrame(index_list)
df_with_additional_info.columns=['index']
df_with_additional_info['disagreement_factor'] = agreements_list
df_with_additional_info['unlabeled_flag'] = unlabeled_list

In [40]:
df_with_additional_info=df_with_additional_info.rename(columns={'index': 'index'})
original_data = DU

In [41]:
df_with_additional_info = df_with_additional_info.merge(original_data, on=['index'], how='left', indicator= True)

In [42]:
cond1 = df_with_additional_info['unlabeled_flag'] == True
cond2 = df_with_additional_info['disagreement_factor'] <= 2
df_active_learning= df_with_additional_info[cond1 | cond2]
print("Data with additional info: disagreements and abstain")
print(df_with_additional_info.shape)
print("Data for Active learning: Data with additional info after applying conditions")
print(df_active_learning.shape)
df_active_learning = df_active_learning.drop(['unlabeled_flag', 'disagreement_factor', '_merge'], axis=1)
df_active_learning['prediction'] = predictions

Data with additional info: disagreements and abstain
(5297, 7)
Data for Active learning: Data with additional info after applying conditions
(5297, 7)


In [43]:
df_with_additional_info

,index,disagreement_factor,unlabeled_flag,At1,At2,ground_truth,_merge
0,1,1,False,1.1400,-0.11400,-1,both
1,2,1,False,-1.5200,-1.15000,1,both
2,3,1,False,-1.0500,0.72000,-1,both
3,4,1,False,-0.9160,0.39700,1,both
4,5,1,False,-1.0900,0.43700,1,both
5,6,1,False,-0.5840,0.09370,1,both
6,7,1,False,1.8300,0.45200,-1,both
7,8,1,False,-1.2500,-0.28600,1,both
8,9,1,False,1.7000,1.21000,1,both
9,10,1,False,-0.4820,-0.48500,1,both


In [44]:
budget_variable = 0.7
labeling_budget = int(budget_variable*len(df_with_additional_info.index))
if(labeling_budget >= df_active_learning.shape[0]):
    labeling_budget= int(1*len(df_active_learning.index))

In [17]:
from sklearn.ensemble import RandomForestRegressor
from data_learner.active_learner import *
from data_learner.models import DDL_Dataset 
from data_learner.lal_model import LALmodel

fn = 'LAL-iterativetree-simulatedunbalanced-big.npz'
parameters = {'est': 1000, 'depth': 40, 'feat': 6 }
filename = '../smartmendr/data_learner/datasets/'+fn
regression_data = np.load(filename)
regression_features = regression_data['arr_0']
regression_labels = regression_data['arr_1']
lalModel = RandomForestRegressor(n_estimators = parameters['est'], max_depth = parameters['depth'], 
                                 max_features=parameters['feat'], oob_score=True, n_jobs=8)
lalModel.fit(regression_features, np.ravel(regression_labels))    

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=8, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [ ]:
from smartmendr.data_learner.dl_utils import *
indices, labels= run_dll(lalModel, labeling_budget,df_active_learning);

[[ 1.000e+00  1.140e+00 -1.140e-01 -1.000e+00]
 [ 2.000e+00 -1.520e+00 -1.150e+00  1.000e+00]
 [ 3.000e+00 -1.050e+00  7.200e-01 -1.000e+00]
 ...
 [ 4.235e+03 -4.210e-01 -1.090e+00 -1.000e+00]
 [ 4.236e+03  8.490e-01  9.070e-01 -1.000e+00]
 [ 4.237e+03 -1.170e+00  1.080e+00 -1.000e+00]]


In [ ]:
AL_Results=pd.DataFrame(columns=['index','AL_Label'])
AL_Results['index']=indices.astype('int64')
AL_Results['AL_Label']=labels
AL_Results = AL_Results.sort_values(by =['index'])

In [ ]:
indices

In [ ]:
AL_Results.loc[AL_Results['AL_Label']==0, 'AL_Label']=-1
data_with_AL_Results = df_with_additional_info.merge(AL_Results, on=['index'], how='left')
true_label = data_with_AL_Results['AL_Label']
true_label = true_label.fillna(0)
for i in range(len(true_label)):
    if true_label[i] !=0:
        L_train[i,:]=true_label[i]
gen_model, AL_train_marginals = Fitting_Gen_Model(L_train)
predictions = gen_model.predictions(reef_L_train, batch_size=None)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(predictions, bins=20)
plt.show()